In [1]:
import pandas as pd
dfs2=pd.read_excel("FinalDatasetAMRResult.xlsx",index_col=0)

In [2]:
dfs=pd.read_excel("FinalDatasetDistilBert.xlsx",index_col=0)
dfs['HighestScorePattern']=dfs2['HighestScorePattern']

In [3]:
dfs

,adj/noun,n_v,prep,pobj,adj/noun-label,pobj-label,adj/noun.1,arg0,V,baseV,...,OVMScore,OVpMScore,VOpMScore,VpOpMScore,VMpOScore,VpMpOScore,MVpOPassiveScore,MpOVPassiveScore,OVpMPassiveScore,HighestScorePattern
0,electoral,participation,of,new parties,arg1,arg0,election,new parties,participate,participate,...,-29.413098,-13.445650,-23.946176,-24.504286,-28.340291,-18.826996,-19.885696,-35.510721,-26.119903,OVpM
1,choral,responses,by,the characters,PP,arg0,chorus,the characters,respond,respond,...,-37.455313,-24.504198,-35.355229,-27.451485,-37.886873,-28.519923,-29.148836,NaN,-32.221162,OVpM
2,cultural,definitions,of,rape,arg0,arg1,culture,culture,defines,define,...,-28.648668,-26.138790,-29.506207,-37.723423,-27.645725,-23.099325,-29.075362,-38.024532,-25.801627,VOpM
3,legal,requirement,for,minimum air-quality standards,arg0,arg1,law,the law,requires,require,...,-33.856446,-29.961117,-29.425627,-32.687595,-36.680362,-36.681547,-27.757004,-43.611185,-22.518502,VOpM
4,financial,evaluation,of,customer referral programs,arg1,arg1,finance,NaN,evaluate,evaluate,...,-27.408240,-31.402770,-26.496691,-32.109442,-29.420844,-30.522685,-23.082835,-38.684745,-29.744935,VOpM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,class,analysis,of,society,arg1,arg1,social class,NaN,analyse,analyse,...,-28.829425,-26.841941,NaN,NaN,-16.646215,-17.922817,-20.188120,-22.117021,NaN,VMpO
445,computer,analysis,of,the text,arg0,arg1,computer,the computer,analyzes,analyze,...,-34.359797,-29.596114,-24.690380,-33.136121,-36.626081,-35.283707,-33.057322,NaN,-22.008722,MVO
446,computer,analysis,of,the text,arg0,arg1,computer,the computer,analyses,analyse,...,-34.443063,-33.935062,-22.599358,-24.053141,-28.276795,-34.061015,-23.877605,NaN,-15.852996,OVpMPassive
447,computer,analysis,of,the text,PP,arg1,computer,NaN,analyze,analyze,...,-34.359797,-29.596114,-24.690380,-33.136121,-36.626081,-35.283707,-33.057322,NaN,-22.008722,MVO


In [4]:
import numpy as np
dfs['arg0']=dfs['arg0'].replace(np.NaN,'')
dfs['arg1']=dfs['arg1'].replace(np.NaN,'')
dfs['PP']=dfs['PP'].replace(np.NaN,'')

In [5]:
columnList=['arg0','baseV','arg1','PP']
findIt=False
for index,i in enumerate(dfs['adj/noun']):
    result=''
    for column in columnList:
        if dfs.loc[index,column]!='':
                result=result+dfs.loc[index,column]+" "
    dfs.loc[index,'gold']=result

In [6]:
for index,i in enumerate(dfs['adj/noun']):
    if index+1<len(dfs['adj/noun']) and dfs.loc[index,'adj/noun']==dfs.loc[index+1,'adj/noun'] and dfs.loc[index,'n_v']==dfs.loc[index+1,'n_v'] and dfs.loc[index,'prep']==dfs.loc[index+1,'prep'] and dfs.loc[index,'pobj']==dfs.loc[index+1,'pobj']:
        dfs.loc[index,'gold2']=dfs.loc[index+1,'gold']
    else:
        dfs.loc[index,'gold2']=dfs.loc[index,'gold']

In [7]:
for index,i in enumerate(dfs['adj/noun']):
    #print(index)
    dfs.loc[index,'OriginalHighestScorePattern']=dfs.loc[index,'HighestScorePattern']
    if dfs.loc[index,'HighestScorePattern']=='MVpOPassive':
        pattern=dfs.loc[index,'HighestScorePattern']
        if "by" in dfs.loc[index,pattern].split():
            dfs.loc[index,'HighestScorePattern']='OVM'
        else:  
            dfs.loc[index,'HighestScorePattern']='MVpOPassiveVMpO'
    if dfs.loc[index,'HighestScorePattern']=='MpOVPassive':
            dfs.loc[index,'HighestScorePattern']='MpOVPassiveVMpO'
    if dfs.loc[index,'HighestScorePattern']=='OVpMPassive':
        pattern=dfs.loc[index,'HighestScorePattern']
        if "by" in dfs.loc[index,pattern].split():
            dfs.loc[index,'HighestScorePattern']='MVO'
        else:
            dfs.loc[index,'HighestScorePattern']='OVpMPassiveVOpM'

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer

In [9]:
for index,i in enumerate(dfs['gold']):
    if not pd.isnull(i):
        doc=nlp(i)
        reducedGold=''
        for word in doc:
            if word.tag_!='DT':
                    reducedGold=reducedGold+word.lemma_+" "
        dfs.loc[index,'reducedGold']=reducedGold.lower()
    reducedGold3=''
    for para in dfs.loc[index,'gold'].split():
        doc=nlp(para)
        for word in doc:
            if word.tag_!='DT':
                if word.text!='a' and word.text!='an' and word.text!='the':
                    answer=lemmatizer(word.text, NOUN)
                    reducedGold3=reducedGold3+answer[0]+" "
    dfs.loc[index,'reducedGold3']=reducedGold3.lower()

In [10]:
for index,i in enumerate(dfs['gold2']):
    if not pd.isnull(i):
        doc=nlp(i)
        reducedGold=''
        for word in doc:
            if word.tag_!='DT':
                reducedGold=reducedGold+word.lemma_+" "
        dfs.loc[index,'reducedGold2']=reducedGold.lower()
    reducedGold4=''
    for para in dfs.loc[index,'gold2'].split():
        doc=nlp(para)
        for word in doc:
            if word.tag_!='DT':
                if word.text!='a' and word.text!='an' and word.text!='the':
                    answer=lemmatizer(word.text, NOUN)
                    reducedGold4=reducedGold4+answer[0]+" "
    dfs.loc[index,'reducedGold4']=reducedGold4.lower()

In [11]:
for index,i in enumerate(dfs['adj/noun']):
    pattern=dfs.loc[index,'HighestScorePattern']
    dfs.loc[index,'HighestScorePatternSentence']=dfs.loc[index,pattern]
    doc=nlp(dfs.loc[index,pattern])
    reducedParaphrase=''
    for word in doc:
        if word.tag_!='DT':
            if word.text==dfs.loc[index,'V']:
                reducedParaphrase=reducedParaphrase+dfs.loc[index,'baseV']+" "
            else:
                reducedParaphrase=reducedParaphrase+word.lemma_+" "
    dfs.loc[index,'reducedParaphrase']=reducedParaphrase.lower()
    reducedParaphrase2=''
    for para in dfs.loc[index,pattern].split():
        doc=nlp(para)
        for word in doc:
            if word.tag_!='DT':
                if word.text!='a' and word.text!='an' and word.text!='the' and word.text!=dfs.loc[index,'V']:
                    answer=lemmatizer(word.text, NOUN)
                    reducedParaphrase2=reducedParaphrase2+answer[0]+" "
                if word.text==dfs.loc[index,'V']:
                    reducedParaphrase2=reducedParaphrase2+dfs.loc[index,'baseV']+" "
    dfs.loc[index,'reducedParaphrase2']=reducedParaphrase2.lower()

In [12]:
for index,i in enumerate(dfs['adj/noun']):
    if (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold2']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold3']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold4']):
        dfs.loc[index,'FinalLabel']='True'
    elif (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold2']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold3']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold4']):
        dfs.loc[index,'FinalLabel']='True'
    else:
        dfs.loc[index,'FinalLabel']='False'

In [13]:
dfs=dfs.fillna('')
for index,i in enumerate(dfs['adj/noun']):
    for columns in dfs.columns:
        dfs.loc[index,columns]=str(dfs.loc[index,columns])
dfs=dfs.groupby(['adj/noun','n_v','prep','pobj'],sort=False)[dfs.columns[4:]].agg('//'.join).reset_index()
dfs['final']=dfs['FinalLabel']
for index,i in enumerate(dfs['adj/noun']):
    if 'True' in dfs.loc[index,'FinalLabel'].split("//"):
        dfs.loc[index,'FinalLabel']='True'
    else:
        dfs.loc[index,'FinalLabel']='False'

In [14]:
dfs['FinalLabel'].value_counts()

False    243
True     126
Name: FinalLabel, dtype: int64

In [15]:
dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel'])

0.34146341463414637

In [16]:
dfs.to_excel("FinalDatasetAMRIgnoreARemoveArticleMetric.xlsx")

In [17]:
dfs2=dfs[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]

In [18]:
import json

with open('FinalDatasetAMRIgnoreARemoveArticleMetric.txt', 'w') as file:
# as requested in comment
    for index,i in zip(dfs2.index,dfs2['sentence']):
        elementDict=dfs2.loc[index].to_dict()
        file.write(str(index)+". ")
        for i in elementDict:
             if elementDict[i]!='':
                 print (i,elementDict[i])
                 file.write(i+" : "+str(elementDict[i]))
                 file.write('\n')
        file.write('\n')

sentence - make the foundation and/ or the electoral participation of new parties practically impossible, because such parties do not receive public funding right from the start and occasionally private funding is prohibited.
adj/noun electoral
n_v participation
prep of
pobj new parties
originalPattern electoral participation of new parties
MVO elections participate new parties
MVOScore -30.27147531509399
MVpO elections participate by new parties
MVpOScore -26.33104634284973
OVM new parties participate elections
OVMScore -29.41309833526611
OVpM new parties participate in elections
OVpMScore -13.4456497207284
VOpM participate new parties in elections
VOpMScore -23.94617629051208
VpOpM participate in new parties in elections
VpOpMScore -24.50428628921509
VMpO participate elections for new parties
VMpOScore -28.34029066562653
VpMpO participate in elections for new parties
VpMpOScore -18.82699552178383
gold new parties participate in elections 
reducedGold new party participate in election

reducedGold judiciary analyze constitutionality of campaign finance restriction //judiciary analyse constitutionality of campaign finance restriction //court analyze constitutionality of campaign finance restriction //court analyse constitutionality of campaign finance restriction 
HighestScorePattern MVO//MVO//MVO//MVO
HighestScorePatternSentence the judiciary analyzes the constitutionality of campaign finance restrictions//the judiciaries analyse the constitutionality of campaign finance restrictions//the court analyzes the constitutionality of campaign finance restrictions//the courts analyse the constitutionality of campaign finance restrictions
reducedParaphrase judiciary analyze constitutionality of campaign finance restriction //judiciary analyse constitutionality of campaign finance restriction //court analyze constitutionality of campaign finance restriction //court analyse constitutionality of campaign finance restriction 
adj/noun-label arg0//arg0//arg0//arg0
FinalLabel True

MVOScore -14.69544458389282
MVpO the theater adapts to the novel
MVpOScore -20.46106934547424
OVM the novel adapts theater
OVMScore -25.91533970832825
OVpM the novel adapts to theater
OVpMScore -14.83975151181221
VOpM adapt the novel into theaters
VOpMScore -14.00877779722214
VpOpM adapt of the novel into theater
VpOpMScore -25.62199974060059
VMpO adapt theater to the novel
VMpOScore -29.05135822296143
VpMpO adapt the theater for the novel
VpMpOScore -30.55752396583557
gold adapt the novel for theater 
reducedGold adapt novel for theater 
HighestScorePattern OVpMPassiveVOpM
HighestScorePatternSentence adapt the novel for theaters
reducedParaphrase adapt novel for theater 
adj/noun-label PP
FinalLabel True
sentence He accepted only two sacraments as valid under the new covenant: baptism and the Lord's Supper (in opposition to the Catholic acceptance of seven sacraments).
adj/noun Catholic
n_v acceptance
prep of
pobj seven sacraments
originalPattern Catholic acceptance of seven sacrament

gold the tribe support the rebellion 
reducedGold tribe support rebellion 
HighestScorePattern VpOpM
HighestScorePatternSentence support for the rebellion against the tribe
reducedParaphrase support for rebellion against tribe 
adj/noun-label arg0
FinalLabel False
sentence In the "Shreveport Rate Cases" (1914), the Court permitted congressional regulation of railroad lines because Congress was regulating the "channels of commerce" and although the regulation was on intrastate rail lines, the effect of the intrastate lines was direct so as to concern interstate commerce.
adj/noun congressional
n_v regulation
prep of
pobj railroad lines
originalPattern congressional regulation of railroad lines
MVO the congress regulates railroad lines
MVOScore -17.33092641830444
MVpO the congress regulates the railroad lines
MVpOScore -21.72841319441795
OVM railroad lines regulate congresses
OVMScore -28.50950741767883
OVpM railroad lines regulate by congress
OVpMScore -25.88631224632263
VOpM regulate r

VpMpOScore -21.39001244306564
gold strike the United States with nuclear weapons 
reducedGold strike united states with nuclear weapon 
HighestScorePattern VpMpO
HighestScorePatternSentence strike against nuclear weapons in the United States
reducedParaphrase strike against nuclear weapon in united states 
adj/noun-label PP
FinalLabel False
sentence On July 17, 2011, Benson was arrested in Austin, Texas for a misdemeanor charge of assault causing bodily injury to a family member .
adj/noun bodily
n_v injury
prep to
pobj a family member
originalPattern bodily injury to a family member
MVO another body injures a family member
MVOScore -19.90218247845769
MVpO the body injures as a family member
MVpOScore -26.78691554069519
OVM a family member injures body
OVMScore -28.59428715705872
OVpM a family member injures the body
OVpMScore -17.53907655645162
VOpM injure a family member in the body
VOpMScore -27.35134315490723
VpOpM injure of a family member in the body
VpOpMScore -37.26949453353882

...

VpOpM tour of Jean Monnet's troupe to London
VpOpMScore -40.95220991957467
VMpO tour London with Jean Monnet's troupe
VMpOScore -42.65793778025545
VpMpO tour of London by Jean Monnet's troupe
VpMpOScore -37.04540817160159
gold Jean Monnet's troupe tour London 
reducedGold jean monnet 's troupe tour london 
HighestScorePattern VpOpM
HighestScorePatternSentence tour of Jean Monnet's troupe to London
reducedParaphrase tour of jean monnet 's troupe to london 
adj/noun-label arg1
FinalLabel False
sentence In the Indian subcontinent, irrigation requirements of Bangladesh determine water allocations of The Ganges River.
adj/noun water
n_v allocations
prep of
pobj The Ganges River
originalPattern water allocations of The Ganges River
MVO the waters allocate The Ganges River
MVOScore -37.44496539235115
MVpO the waters allocate to The Ganges River
MVpOScore -36.10466701164842
OVM The Ganges River allocates water
OVMScore -23.07546793855727
OVpM The Ganges River allocates the waters
OVpMScore -27

VOpMScore -20.65304690599442
VpOpM discharge from the sea into the river
VpOpMScore -15.25218790769577
VMpO discharge rivers into the sea
VMpOScore -19.15813565254211
VpMpO discharge of the river into the sea
VpMpOScore -14.2335556447506
gold the river discharge into the sea 
reducedGold river discharge into sea 
HighestScorePattern MVpO
HighestScorePatternSentence the river discharges into the sea
reducedParaphrase river discharge into sea 
adj/noun-label arg0
FinalLabel True
sentence The UNHCR, which mainly partners with the Syrian Red Crescent, also runs a number of educational, professional development, and psychosocial support programs; it additionally offers food, critical household goods, and direct cash assistance to some refugees.
adj/noun cash
n_v assistance
prep to
pobj some refugees
originalPattern cash assistance to some refugees
MVO cash assists some refugees
MVOScore -32.75139570236206
MVpO cash assists for some refugees
MVpOScore -35.83798062801361
OVM some refugees ass

HighestScorePatternSentence analyze the imperialist relations of the great world powers during wartime//analyse the imperialist relations of the great world powers during wartime//analyze the imperialist relations of the great world powers during wartime//analyse the imperialist relations of the great world powers during wartime
reducedParaphrase analyze imperialist relation of great world power during wartime //analyse imperialist relation of great world power during wartime //analyze imperialist relation of great world power during wartime //analyse imperialist relation of great world power during wartime 
adj/noun-label PP//PP//PP//PP
FinalLabel True
sentence Since each case is different, court interpretations of the statutory law gets evolved and have either narrowed or widened their scope.
adj/noun court
n_v interpretations
prep of
pobj the statutory law
originalPattern court interpretations of the statutory law
MVO the court interprets the statutory law
MVOScore -17.2404445707798

pobj the regiment
originalPattern dress distinctions of the regiment
MVO this dress distinguishes the regiment
MVOScore -21.53068799339235
MVpO this dress distinguishes from the regiment
MVpOScore -25.21883344650269
OVM the regiment distinguishes dresses
OVMScore -34.49689364433289
OVpM the regiment distinguishes the dress
OVpMScore -27.90428764373064
VOpM distinguish the regiment from the dress
VOpMScore -30.31725916266441
VpOpM distinguish between the regiment by the dress
VpOpMScore -37.01764941215515
VMpO distinguish dress of the regiment
VMpOScore -29.16545736789703
VpMpO distinguish between the dress of the regiment
VpMpOScore -26.94748508930206
gold distinguish the regiment by dress 
reducedGold distinguish regiment by dress 
HighestScorePattern VpMpO
HighestScorePatternSentence distinguish between the dress of the regiment
reducedParaphrase distinguish between dress of regiment 
adj/noun-label PP
FinalLabel False
sentence Apart from practical implications of this, especially if

VMpO distribute home for anime releases//distribute home for anime releases
VMpOScore -37.69019508361816//-37.69019508361816
VpMpO distribute at home for anime releases//distribute at home for anime releases
VpMpOScore -30.69074058532715//-30.69074058532715
gold distribute anime releases to homes //distribute anime releases in homes 
reducedGold distribute anime release to home //distribute anime release in home 
HighestScorePattern VOpM//VOpM
HighestScorePatternSentence distribute anime releases at home//distribute anime releases at home
reducedParaphrase distribute anime release at home //distribute anime release at home 
adj/noun-label PP//PP
FinalLabel False
sentence Witt played Kriemhild in this film interpretation of the epic poem "Das Nibelungenlied".//Witt played Kriemhild in this film interpretation of the epic poem "Das Nibelungenlied".
adj/noun film
n_v interpretation
prep of
pobj the epic poem
originalPattern film interpretation of the epic poem//film interpretation of the 

In [19]:
dfs3=dfs[dfs['FinalLabel']=='False']

In [20]:
dfs3=dfs3[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]

In [21]:
import json

with open('FinalDatasetAMRIgnoreARemoveArticleMetricFalse.txt', 'w') as file:
# as requested in comment
    for index,i in zip(dfs3.index,dfs3['sentence']):
        elementDict=dfs3.loc[index].to_dict()
        file.write(str(index)+". ")
        for i in elementDict:
             if elementDict[i]!='':
                 print (i,elementDict[i])
                 file.write(i+" : "+str(elementDict[i]))
                 file.write('\n')
        file.write('\n')

sentence A chorus illuminates the significance of events and is told to "sing . . . after the manner of choruses in oratorios." (The work also uses a peasant chorus and some choral responses by the characters .) The music is to be "functional" and to include fanfares and alarm bells.
adj/noun choral
n_v responses
prep by
pobj the characters
originalPattern choral responses by the characters
MVO the chorus responds the characters
MVOScore -30.72011184692383
MVpO the chorus responds to the characters
MVpOScore -25.67723596096039
OVM the characters respond choruses
OVMScore -37.45531320571899
OVpM the characters respond to these choruses
OVpMScore -24.5041977763176
VOpM respond the characters in the chorus
VOpMScore -35.35522902011871
VpOpM respond to the characters in the chorus
VpOpMScore -27.45148450136185
VMpO respond choruses by the characters
VMpOScore -37.88687252998352
VpMpO respond to the chorus of the characters
VpMpOScore -28.5199226140976
gold the characters respond in chorus 

HighestScorePattern VpOpM
HighestScorePatternSentence explain the ice ages in astronomy
reducedParaphrase explain ice age in astronomy 
adj/noun-label arg0
FinalLabel False
sentence However, chemical analysis of Martian meteorite samples suggests that the ambient near-surface temperature of Mars has most likely been below 0 °C for the last four billion years.//However, chemical analysis of Martian meteorite samples suggests that the ambient near-surface temperature of Mars has most likely been below 0 °C for the last four billion years.
adj/noun chemical
n_v analysis
prep of
pobj Martian meteorite samples
originalPattern chemical analysis of Martian meteorite samples//chemical analysis of Martian meteorite samples
MVO chemical properties analyze Martian meteorite samples//chemical properties analyse Martian meteorite samples
MVOScore -29.93953171791509//-24.88674213533523
MVpO chemical properties analyze in Martian meteorite samples//chemical property analyses of Martian meteorite samp

HighestScorePatternSentence strike against nuclear weapons in the United States
reducedParaphrase strike against nuclear weapon in united states 
adj/noun-label PP
FinalLabel False
sentence One example of parental response to children 's behavior is the elicitation of scaffolding behavior, which in turn affects the level of behavior children show in the future.
adj/noun parental
n_v response
prep to
pobj children 's behavior
originalPattern parental response to children 's behavior
MVO a parent responds children 's behavior
MVOScore -28.35211516311392
MVpO a parent responds to children 's behavior
MVpOScore -14.0991545910947
OVM children 's behavior responds parents
OVMScore -34.98024637519848
OVpM children 's behavior responds to parents
OVpMScore -23.00369678018615
VOpM respond children 's behavior to parents
VOpMScore -34.515352956485
VpOpM respond to children 's behavior with parents
VpOpMScore -21.88494672207162
VMpO respond parents to children 's behavior
VMpOScore -30.4329387904

FinalLabel False
sentence The two men fight with Kato easily defeating Bae-dal, but an American attack on the airforce base allows Bae-dal and Chun-bae to escape.//The two men fight with Kato easily defeating Bae-dal, but an American attack on the airforce base allows Bae-dal and Chun-bae to escape.
adj/noun American
n_v attack
prep on
pobj the airforce base
originalPattern American attack on the airforce base//American attack on the airforce base
MVO an American attacks the airforce base//America attacks the airforce base
MVOScore -32.00857651233673//-32.0886070728302
MVpO American attacks on the airforce base//America attacks on the airforce base
MVpOScore -21.51739272475243//-30.49564945697784
OVM the airforce base attacks American//the airforce base attacks America
OVMScore -38.42641949653625//-37.65831279754639
OVpM the airforce base attacks against Americans//the airforce base attacks the Americas
OVpMScore -38.07776939868927//-40.44992709159851
VOpM attack the airforce base on t

MVOScore -27.59057489037514
MVpO a territory acquires from a dynasty
MVpOScore -29.40217387676239
OVM a dynasty acquires territories
OVMScore -25.85878169536591
OVpM a dynasty acquires a territory
OVpMScore -26.66933937370777
VOpM acquire a dynasty with territories
VOpMScore -33.63178205490112
VpOpM acquire into a dynasty of territories
VpOpMScore -36.46213710308075
VMpO acquire territories within a dynasty
VMpOScore -28.2974499464035
VpMpO acquire the territories of a dynasty
VpMpOScore -29.75848203897476
gold the dynasty acquire territories 
reducedGold dynasty acquire territory 
HighestScorePattern VMpO
HighestScorePatternSentence acquire territories within a dynasty
reducedParaphrase acquire territory within dynasty 
adj/noun-label arg1
FinalLabel False
sentence Judith Herman suggests, in "Trauma and Recovery: The Aftermath of Violence--from Domestic Abuse to Political Terror", that hysteria was a label given a form of PTSD arising from routine domestic abuse of women, including ph

...

sentence Presentations include experimental evaluation of spray systems and spray nozzles under a variety of applications.
adj/noun experimental
n_v evaluation
prep of
pobj spray systems
originalPattern experimental evaluation of spray systems
MVO experiments evaluate spray systems
MVOScore -39.20173311233521
MVpO an experiment evaluates the spray systems
MVpOScore -38.24637711048126
OVM spray systems evaluate experiments
OVMScore -36.88056564331055
OVpM spray systems evaluate each experiment
OVpMScore -36.98739147186279
VOpM evaluate spray systems in an experiment
VOpMScore -42.89304375648499
VpOpM evaluate the spray systems in an experiment
VpOpMScore -39.76196420192719
VMpO evaluate experiments with spray systems
VMpOScore -36.34626340866089
VpMpO evaluate an experiment with spray systems
VpMpOScore -39.33929967880249
gold evaluate spray systems through experiment 
reducedGold evaluate spray system through experiment 
HighestScorePattern OVpMPassiveVOpM
HighestScorePatternSentence e

VOpMScore -41.88938879966736
VpOpM distinguish between mortal and venial sin from Catholics
VpOpMScore -39.67673122882843
VMpO distinguish Catholics between mortal and venial sin
VMpOScore -39.68248215317726
VpMpO distinguish the Catholics between mortal and venial sin
VpMpOScore -43.13041011989117
gold Catholics distinguish between mortal and venial sin 
reducedGold catholics distinguish between mortal and venial sin 
HighestScorePattern MVpO
HighestScorePatternSentence the Catholic distinguishes both mortal and venial sin
reducedParaphrase catholic distinguishe both mortal and venial sin 
adj/noun-label arg0
FinalLabel False
sentence Federal Labor has also committed to further road improvements in the area through the extension of the Monaro Highway.
adj/noun road
n_v improvements
prep in
pobj the area
originalPattern road improvements in the area
MVO the road improves the area
MVOScore -20.56636942178011
MVpO the road improves throughout the area
MVpOScore -20.01490795612335
OVM the

adj/noun-label arg1
FinalLabel False
sentence Waste and sewage are two major problems for cities, as is air pollution from various forms of combustion, including fireplaces, wood or coal-burning stoves, other heating systems, and internal combustion engines.
adj/noun air
n_v pollution
prep from
pobj various forms of combustion
originalPattern air pollution from various forms of combustion
MVO the air pollutes various forms of combustion
MVOScore -17.1476362273097
MVpO air pollutes from various forms of combustion
MVpOScore -17.38112731650472
OVM various forms of combustion pollutes air
OVMScore -27.77537971735001
OVpM various forms of combustion pollutes the air
OVpMScore -17.10917105898261
VOpM pollute various forms of combustion into the air
VOpMScore -23.11902856826782
VpOpM pollute the various forms of combustion of air
VpOpMScore -24.62662729620934
VMpO pollute air in various forms of combustion
VMpOScore -18.36907389760017
VpMpO pollute the air through various forms of combustion

sentence Despite the off-season acquisitions of several players, such as Shannon Byrnes, Tom Gillies, Chris Dawes, David Rodan and Jack Viney, the Demons underachieved in the first half of the 2013 season, managing just one win in their first 11 games.//Despite the off-season acquisitions of several players, such as Shannon Byrnes, Tom Gillies, Chris Dawes, David Rodan and Jack Viney, the Demons underachieved in the first half of the 2013 season, managing just one win in their first 11 games.
adj/noun off-season
n_v acquisitions
prep of
pobj several players
originalPattern off-season acquisitions of several players//off-season acquisitions of several players
MVO an off season acquires several players//an off season acquires several players
MVOScore -23.17072546482086//-23.17072546482086
MVpO an off season acquires a several players//an off season acquires a several players
MVpOScore -35.32735955715179//-35.32735955715179
OVM several players acquire off season//several players acquire o

sentence In 2006, Corona Power, LLC sought and won a substantial reduction in the tax assessment of the property.//In 2006, Corona Power, LLC sought and won a substantial reduction in the tax assessment of the property.
adj/noun tax
n_v assessment
prep of
pobj the property
originalPattern tax assessment of the property//tax assessment of the property
MVO a tax assesses the property//a tax assesses the property
MVOScore -20.68814736604691//-20.68814736604691
MVpO the tax assesses on the property//the tax assesses on the property
MVpOScore -25.12293839454651//-25.12293839454651
OVM the property assesses taxes//the property assesses taxes
OVMScore -23.64342677593231//-23.64342677593231
OVpM the property assesses a tax//the property assesses a tax
OVpMScore -23.34002703428268//-23.34002703428268
VOpM assess the property without tax//assess the property without tax
VOpMScore -32.13043737411499//-32.13043737411499
VpOpM assess on the property without tax//assess on the property without tax
V

In [22]:
dfs4=dfs[dfs['FinalLabel']=='True']

In [23]:
dfs4=dfs4[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]

In [24]:
import json

with open('FinalDatasetAMRIgnoreARemoveArticleMetricTrue.txt', 'w') as file:
# as requested in comment
    for index,i in zip(dfs4.index,dfs4['sentence']):
        elementDict=dfs4.loc[index].to_dict()
        file.write(str(index)+". ")
        for i in elementDict:
             if elementDict[i]!='':
                 print (i,elementDict[i])
                 file.write(i+" : "+str(elementDict[i]))
                 file.write('\n')
        file.write('\n')

sentence - make the foundation and/ or the electoral participation of new parties practically impossible, because such parties do not receive public funding right from the start and occasionally private funding is prohibited.
adj/noun electoral
n_v participation
prep of
pobj new parties
originalPattern electoral participation of new parties
MVO elections participate new parties
MVOScore -30.27147531509399
MVpO elections participate by new parties
MVpOScore -26.33104634284973
OVM new parties participate elections
OVMScore -29.41309833526611
OVpM new parties participate in elections
OVpMScore -13.4456497207284
VOpM participate new parties in elections
VOpMScore -23.94617629051208
VpOpM participate in new parties in elections
VpOpMScore -24.50428628921509
VMpO participate elections for new parties
VMpOScore -28.34029066562653
VpMpO participate in elections for new parties
VpMpOScore -18.82699552178383
gold new parties participate in elections 
reducedGold new party participate in election

sentence In musical theatre and opera, the music director is in charge of the overall musical performance, including ensuring that the cast knows the music thoroughly, supervising the musical interpretation of the performers and pit orchestra, and conducting the orchestra.
adj/noun musical
n_v interpretation
prep of
pobj the performers
originalPattern musical interpretation of the performers
MVO the music interprets the performers
MVOScore -28.18416583538055
MVpO the music interprets from the performers
MVpOScore -31.76261496543884
OVM the performers interpret music
OVMScore -24.57167482376099
OVpM the performers interpret the music
OVpMScore -21.05533480644226
VOpM interpret the performers the music
VOpMScore -31.70814323425293
VpOpM interpret by the performers the music
VpOpMScore -34.13948059082031
VMpO interpret music for the performers
VMpOScore -30.10223340988159
VpMpO interpret the music by the performers
VpMpOScore -23.59751522541046
gold the performers interpret the music 
red

sentence A rare example of American influence in Danish architecture , the building now serves as an exhibition space.
adj/noun American
n_v influence
prep in
pobj Danish architecture
originalPattern American influence in Danish architecture
MVO America influences Danish architecture
MVOScore -30.63772201538086
MVpO America influences the Danish architecture
MVpOScore -35.12324905395508
OVM Danish architecture influences America
OVMScore -35.0699667930603
OVpM Danish architecture influences throughout the Americas
OVpMScore -28.88939225673676
VOpM influence Danish architecture throughout the Americas
VOpMScore -32.42959403991699
VpOpM influence of Danish architecture throughout the Americas
VpOpMScore -29.77767211198807
VMpO influence America in Danish architecture
VMpOScore -42.07244324684143
VpMpO influence on America by Danish architecture
VpMpOScore -34.48558104038239
gold America influence Danish architecture 
reducedGold america influence danish architecture 
HighestScorePattern 

MVpO performances evaluate in the business
MVpOScore -39.59943437576294
OVM the business evaluates performance
OVMScore -23.59927022457123
OVpM the business evaluates the performance
OVpMScore -24.19442321360111
VMpO evaluate performance within the business
VMpOScore -21.00196301937103
VpMpO evaluate the performance of the business
VpMpOScore -17.14800344407558
gold evaluate the performance of the business 
reducedGold evaluate performance of business 
HighestScorePattern VpMpO
HighestScorePatternSentence evaluate the performance of the business
reducedParaphrase evaluate performance of business 
adj/noun-label arg1
FinalLabel True
sentence Attempts have been made to model the developmental process of language acquisition in children from a computational angle, leading to both statistical grammars and connectionist models.
adj/noun language
n_v acquisition
prep in
pobj children
originalPattern language acquisition in children
MVO languages acquire children
MVOScore -25.54093551635742
M

VOpM analyze Rita in character//analyse Rita in character
VOpMScore -39.42935466766357//-30.81358281522989
VpOpM analyze the Rita of the character//analyse the Rita of the character
VpOpMScore -40.71342098712921//-33.85893137217499
VMpO analyze characters from Rita//analyse characters from Rita
VMpOScore -38.07834219932556//-25.95317403413355
VpMpO analyze the character of Rita//analyse the character of Rita
VpMpOScore -34.8239027261734//-25.97881670831703
gold analyze the character of Rita //analyse the character of Rita 
reducedGold analyze character of rita //analyse character of rita 
HighestScorePattern VpMpO//VMpO
HighestScorePatternSentence analyze the character of Rita//analyse characters from Rita
reducedParaphrase analyze character of rita //analyse character from rita 
adj/noun-label arg1//arg1
FinalLabel True
sentence After this the legislation did prevent a cash distribution to members of less than two years standing, but the same result was obtained by permitting the issu

reducedGold access community via river 
HighestScorePattern OVpMPassiveVOpM
HighestScorePatternSentence access the community via river
reducedParaphrase access community via river 
adj/noun-label PP
FinalLabel True
sentence The scholarly detective work of Douglass Adair in 1944 postulated the following assignments of authorship, corroborated in 1964 by a computer analysis of the text://The scholarly detective work of Douglass Adair in 1944 postulated the following assignments of authorship, corroborated in 1964 by a computer analysis of the text://The scholarly detective work of Douglass Adair in 1944 postulated the following assignments of authorship, corroborated in 1964 by a computer analysis of the text://The scholarly detective work of Douglass Adair in 1944 postulated the following assignments of authorship, corroborated in 1964 by a computer analysis of the text:
adj/noun computer
n_v analysis
prep of
pobj the text
originalPattern computer analysis of the text//computer analysis